In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import pandas as pd
import numpy as np
import datetime
import copy

生成测试数据

In [2]:
data = {"站号":np.arange(54511,54515).tolist(),
        "经度":np.arange(100.0,104,1).tolist(),
        "纬度":np.arange(30.0,34,1).tolist(),
        "数据":(np.random.rand(4)*100).astype(np.int16)}
df = pd.DataFrame(data)
print(df)

      站号     经度    纬度  数据
0  54511  100.0  30.0  54
1  54512  101.0  31.0  45
2  54513  102.0  32.0  26
3  54514  103.0  33.0  63


In [3]:
sta_ob = meb.sta_data(df,columns = ["id","lon","lat","data0"])
meb.set_stadata_names(sta_ob,["ob"])
meb.set_stadata_coords(sta_ob,level = 0,time = datetime.datetime(2019,1,2,8,0),dtime = 0)
print(sta_ob)

   level                time  dtime     id    lon   lat    ob
0      0 2019-01-02 08:00:00      0  54511  100.0  30.0  54.0
1      0 2019-01-02 08:00:00      0  54512  101.0  31.0  45.0
2      0 2019-01-02 08:00:00      0  54513  102.0  32.0  26.0
3      0 2019-01-02 08:00:00      0  54514  103.0  33.0  63.0


In [4]:
sta_ec = copy.deepcopy(sta_ob)
meb.set_stadata_names(sta_ec,["ec"])
sta_ec["time"].values[0:2] = datetime.datetime(2019,1,1,8,0)
sta_ec["time"].values[2:] = datetime.datetime(2019,1,1,20,0)
sta_ec["dtime"].values[0:4:2] = 12
sta_ec["dtime"].values[1:4:2] = 24
print(sta_ec)

   level                time  dtime     id    lon   lat    ec
0      0 2019-01-01 08:00:00     12  54511  100.0  30.0  54.0
1      0 2019-01-01 08:00:00     24  54512  101.0  31.0  45.0
2      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0
3      0 2019-01-01 20:00:00     24  54514  103.0  33.0  63.0


In [5]:
sta_grapes = copy.deepcopy(sta_ec)
meb.set_stadata_names(sta_grapes,["grapes"])
sta_grapes["dtime"].values[8:] = 24
print(sta_grapes)

   level                time  dtime     id    lon   lat  grapes
0      0 2019-01-01 08:00:00     12  54511  100.0  30.0    54.0
1      0 2019-01-01 08:00:00     24  54512  101.0  31.0    45.0
2      0 2019-01-01 20:00:00     12  54513  102.0  32.0    26.0
3      0 2019-01-01 20:00:00     24  54514  103.0  33.0    63.0


# 站点数据纵向拼接  
**combine_join(sta, sta1)**   
将两个站点数据在纵向拼接，拼接后行数会增加


**参数说明：**  
 **sta**: 站点数据  
 **sta1**: 站点数据1         
 **return**: 如果sta和sta1中一个为None，则返回另一个。否则返回两者的拼接结果，拼接后数据名称采用sta中的数据名称      

**调用示例**


In [6]:
sta_c0 = meb.combine_join(None,sta_ob)
print(sta_c0)

   level                time  dtime     id    lon   lat    ob
0      0 2019-01-02 08:00:00      0  54511  100.0  30.0  54.0
1      0 2019-01-02 08:00:00      0  54512  101.0  31.0  45.0
2      0 2019-01-02 08:00:00      0  54513  102.0  32.0  26.0
3      0 2019-01-02 08:00:00      0  54514  103.0  33.0  63.0


In [7]:
sta_c1 = meb.combine_join(sta_ob,sta_ec)
print(sta_c1)

   level                time  dtime     id    lon   lat    ob
0      0 2019-01-02 08:00:00      0  54511  100.0  30.0  54.0
1      0 2019-01-02 08:00:00      0  54512  101.0  31.0  45.0
2      0 2019-01-02 08:00:00      0  54513  102.0  32.0  26.0
3      0 2019-01-02 08:00:00      0  54514  103.0  33.0  63.0
4      0 2019-01-01 08:00:00     12  54511  100.0  30.0  54.0
5      0 2019-01-01 08:00:00     24  54512  101.0  31.0  45.0
6      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0
7      0 2019-01-01 20:00:00     24  54514  103.0  33.0  63.0


# 基于站点的横向合并  
**combine_on_id(sta, sta1)**   
将两个站点数据在横向合并，只保留一份时空坐标信息,站号一致的会被合并到一行。在数据中一个站号对应多条不同时刻或层次的数据记录时，该函数不可用  


**参数说明：**  
 **sta**: 站点数据  
 **sta1**: 站点数据1         
 **return**: 如果sta和sta1中一个为None，则返回另一个。否则返回两者合并结果，合并时采用id作为合并依据，保留两个站点数据中都有的站点，其它时空坐标信息采用sta中的坐标信息。        

**调用示例**

In [8]:
sta_c2 = meb.combine_on_id(sta_ob,sta_ec)
print(sta_c2)

   level                time  dtime     id    lon   lat    ob    ec
0      0 2019-01-02 08:00:00      0  54511  100.0  30.0  54.0  54.0
1      0 2019-01-02 08:00:00      0  54512  101.0  31.0  45.0  45.0
2      0 2019-01-02 08:00:00      0  54513  102.0  32.0  26.0  26.0
3      0 2019-01-02 08:00:00      0  54514  103.0  33.0  63.0  63.0


# 基于所有坐标的横向合并  
**combine_on_all_coords(sta, sta1)**   
将两个站点数据在横向合并，只保留一份时空坐标信息。合并时包括level,time,dtime,id,lon,lat在内的所有坐标都一致的数据才会合并到一行   


**参数说明：**  
 **sta**: 站点数据  
 **sta1**: 站点数据1         
 **return**: 如果sta和sta1中一个为None，则返回另一个。否则返回两者合并结果，合并时仅保留所有维度的取值都一样的站点          

**调用示例**

In [9]:
sta_c3 = meb.combine_on_all_coords(sta_ob,sta_ec)
print(sta_c3)

Empty DataFrame
Columns: [level, time, dtime, id, lon, lat, ob, ec]
Index: []


In [10]:
sta_c4 = meb.combine_on_all_coords(sta_ec,sta_grapes)
print(sta_c4)

   level                time  dtime     id    lon   lat    ec  grapes
0      0 2019-01-01 08:00:00     12  54511  100.0  30.0  54.0    54.0
1      0 2019-01-01 08:00:00     24  54512  101.0  31.0  45.0    45.0
2      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0    26.0
3      0 2019-01-01 20:00:00     24  54514  103.0  33.0  63.0    63.0


# 基于level,time,dtime和id的横向合并
**combine_on_level_time_dtime_id(sta, sta1)**  
将两个站点数据在横向合并，只保留一份时空坐标信息。合并时包括level,time,dtime,id在内的坐标都一致的数据才会合并到一行，该函数的应用场景是有时站点数据中的Id是可靠的，但lon 和lat 不够可靠或不够精确时，该函数能够比combine_on_all_coords实现更准确的匹配。  但是在一些海洋观测中，很多移动观测数据的id不可靠，则不能采用该函数进行数据的合并。 

**参数说明：**  
 **sta**: 站点数据  
 **sta1**: 站点数据1         
 **return**: 如果sta和sta1中一个为None，则返回另一个。否则返回两者合并结果，合并时仅保留level,time,dtime,id共4个坐标取值都一样的站点  
   

**调用示例**

In [11]:
sta_c4 = meb.combine_on_level_time_dtime_id(sta_ec,sta_grapes)
print(sta_c4)

   level                time  dtime     id    lon   lat    ec  grapes
0      0 2019-01-01 08:00:00     12  54511  100.0  30.0  54.0    54.0
1      0 2019-01-01 08:00:00     24  54512  101.0  31.0  45.0    45.0
2      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0    26.0
3      0 2019-01-01 20:00:00     24  54514  103.0  33.0  63.0    63.0


# 观测和预报的匹配合并   
**combine_on_obTime(sta_ob, sta_fo_list,need_match_ob = False)**   
首先不同的预报数据基于所有时空坐标进行横向合并。再根据合并结果中每一行数据中的起报时间（time）和预报时效(dtime)计算出其在观测数据sta_ob中对应的观测时刻ob_time，如果ob_time 和sta_ob中数据时刻一致，且id,lon,lat,level也一致，则将观测值加入到该行数据中的第一列。 


**参数说明：**  
 **sta_ob**: 观测站点数据,其中dtime列都为0，  
 **sta_fo_list**: 预报站点数据列表         
 **need_match_ob**:是否要求在观测不缺测的情况下匹配合并,缺省情况下如果预报对应的观测不存在则会以999999填充，如果该参数选为True，则预报找不到对应的观测会被删除。  
 **return**: 合并后的站点数据。     
 
 

# 观测和预报的匹配合并(水平坐标基于id)   
**combine_on_obTime_id(sta_ob, sta_fo_list,need_match_ob = False)**   
首先不同的预报数据基于id,level,time,dtime进行横向合并。再根据合并结果中每一行数据中的起报时间（time）和预报时效(dtime)计算出其在观测数据sta_ob中对应的观测时刻obTime，如果obTime 和sta_ob中数据时刻一致，且id,level也一致，则将观测值加入到该行数据中的第一列。 


**参数说明：**  
 **sta_ob**: 观测站点数据,其中dtime列都为0，  
 **sta_fo_list**: 预报站点数据列表
 **need_match_ob**:是否要求在观测不缺测的情况下匹配合并,缺省情况下如果预报对应的观测不存在则会以999999填充，如果该参数选为True，则预报找不到对应的观测会被删除。  
 **return**: 合并后的站点数据。          
 
     

**调用示例**

In [12]:
sta_ec.loc[1,"lon"] = 101.1
sta_c4 = meb.combine_on_obTime_id(sta_ob,[sta_ec,sta_grapes])   #保留观测缺测的部分
print(sta_c4)

   level                time  dtime     id    lon   lat        ob    ec  \
0      0 2019-01-01 08:00:00     24  54512  101.1  31.0      45.0  45.0   
1      0 2019-01-01 20:00:00     12  54513  102.0  32.0      26.0  26.0   
2      0 2019-01-01 08:00:00     12  54511  100.0  30.0  999999.0  54.0   
3      0 2019-01-01 20:00:00     24  54514  103.0  33.0  999999.0  63.0   

   grapes  
0    45.0  
1    26.0  
2    54.0  
3    63.0  


In [15]:
sta_c4 = meb.combine_on_obTime_id(sta_ob,[sta_ec,sta_grapes],True)   # 不保留观测缺测的部分
print(sta_c4)

   level                time  dtime     id    lon   lat    ob    ec  grapes
0      0 2019-01-01 08:00:00     24  54512  101.0  31.0  45.0  45.0    45.0
1      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0  26.0    26.0


In [16]:
sta_c4 = meb.combine_on_obTime(sta_ob,[sta_ec,sta_grapes])  
print(sta_c4) #sta_ec站号为54512的站点lon和sta_ob，sta_grapes中有所偏差，采用该函数就无法实现匹配  

   level                time  dtime     id    lon   lat        ob    ec  \
0      0 2019-01-01 20:00:00     12  54513  102.0  32.0      26.0  26.0   
1      0 2019-01-01 08:00:00     12  54511  100.0  30.0  999999.0  54.0   
2      0 2019-01-01 20:00:00     24  54514  103.0  33.0  999999.0  63.0   

   grapes  
0    26.0  
1    54.0  
2    63.0  


In [17]:
sta_c4 = meb.combine_on_obTime(sta_ob,[sta_ec,sta_grapes],True)  
print(sta_c4) #sta_ec站号为54512的站点lon和sta_ob，sta_grapes中有所偏差，采用该函数就无法实现匹配  

   level                time  dtime     id    lon   lat    ob    ec  grapes
0      0 2019-01-01 20:00:00     12  54513  102.0  32.0  26.0  26.0    26.0
